In [1]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image

In [2]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = './TinyImageNet'

In [3]:
label_num = 100

class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 10000

    def __getitem__(self, index):
        img = Image.open(f'{file_path}/test/{index}.jpg')
        # img.show()
        img = my_transform(img)
        # print(img.size())
        # print(aaa)
        return index, img

    def __len__(self):
        return self.length

In [6]:
test_loader = data.DataLoader(ImageSet(), batch_size=10, shuffle=False)

In [7]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = models.resnet18()
convNet.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
convNet.maxpool = nn.Sequential()
convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(256,label_num)
convNet.load_state_dict(torch.load('./ConvNet-100.model'))
convNet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

In [8]:
convNet.eval()
index_list = torch.zeros((0)).int()
predict_list = torch.zeros((0)).int()
for i, (index, images) in enumerate(test_loader):
    outputs = convNet(images)
    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    print(f"i = {i}, index = {index} predict = {predict}")
    index_list = torch.cat((index_list,index))
    predict_list = torch.cat((predict_list,predict))
    # print(index_list)
    # print(predict_list)

i = 0, index = tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) predict = tensor([60, 61, 96, 83, 13, 64, 56, 75, 71, 89])
i = 1, index = tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]) predict = tensor([72, 25, 56, 19, 34, 38, 68, 88, 61,  4])
i = 2, index = tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]) predict = tensor([69, 38, 82, 75, 61, 19, 39, 54, 98, 75])
i = 3, index = tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39]) predict = tensor([94, 39, 22, 35, 69, 32, 33, 54, 38, 96])
i = 4, index = tensor([40, 41, 42, 43, 44, 45, 46, 47, 48, 49]) predict = tensor([31, 61, 75, 82, 75, 69, 23, 61, 82, 38])
i = 5, index = tensor([50, 51, 52, 53, 54, 55, 56, 57, 58, 59]) predict = tensor([69, 73, 89, 59, 40, 67, 58, 67, 84, 38])
i = 6, index = tensor([60, 61, 62, 63, 64, 65, 66, 67, 68, 69]) predict = tensor([69, 49, 38, 27, 79, 12, 38, 94, 75, 69])
i = 7, index = tensor([70, 71, 72, 73, 74, 75, 76, 77, 78, 79]) predict = tensor([97, 38, 43, 43,  5, 38, 72, 39, 75, 38])
i = 8, index = tensor([80,

In [11]:
import pandas as pd
img_list = [f'{i}.jpg' for i in index_list]
dataframe = pd.DataFrame({'Id':img_list,'Category':predict_list})
dataframe.to_csv("test.csv",index=False,sep=',')